为了将 **无松弛变量的定制惩罚函数方法**（如 Lee & Lau 论文中提出的指数或 Heaviside 阶跃惩罚函数）应用于您提供的文章中的 Flexible Open Shop Scheduling Problem (FOSSP)，并结合文章中的 One-hot 编码，替换原有的不等式优化方法（基于松弛变量的动态期望优化），我们将设计一种基于变分量子本征求解器（VQE）的解决方案。以下是详细的解决方法，保持与您文章的上下文一致，并提供完整的实现流程和伪代码。

---

### 1. FOSSP 问题概述与重构
FOSSP 目标是最小化最大完成时间 \( C_{\max } \)，涉及 \( n \) 个作业 \( o_0, o_1, \ldots, o_{n-1} \) 和 \( m \) 台机器 \( M_0, M_1, \ldots, M_{m-1} \)，作业 \( o_i \) 在机器 \( M_j \) 上的处理时间为 \( p_{ij} \)。使用 One-hot 编码，定义二进制变量 \( x_{ij} \in \{0, 1\} \)，其中 \( x_{ij} = 1 \) 表示作业 \( o_i \) 分配到机器 \( M_j \)，约束为：
\[
\sum_{j=0}^{m-1} x_{ij} = 1, \quad \forall i = 0, 1, \ldots, n-1
\]
每台机器的完成时间为：
\[
C_j = \sum_{i=0}^{n-1} x_{ij} p_{ij}, \quad \forall j = 0, 1, \ldots, m-1
\]
目标是：
\[
\min C_{\max }, \quad \text{s.t.} \quad C_j \leq C_{\max }, \quad \forall j
\]
使用 \( l \)-位二进制变量表示 \( C_{\max } \)：
\[
C_{\max } = \sum_{h=0}^{l-1} 2^h z_h, \quad z_h \in \{0, 1\}
\]

原文章使用松弛变量 \( \tau_{jh} \) 和动态期望优化处理不等式约束 \( C_j \leq C_{\max } \)。我们将用无松弛变量的定制惩罚函数方法（以 Heaviside 阶跃函数为主，兼顾指数函数）替换此部分，保持 One-hot 编码。

---

### 2. 无松弛变量的定制惩罚函数方法
Lee & Lau 的方法通过非线性惩罚函数（如 Heaviside 阶跃函数 \( \delta(h_j) \) 或指数函数 \( e^{\lambda h_j} \））直接编码不等式约束，无需松弛变量。对于 FOSSP 的约束：
\[
h_j(x, z) = C_j - C_{\max } = \sum_{i=0}^{n-1} x_{ij} p_{ij} - \sum_{h=0}^{l-1} 2^h z_h \leq 0
\]
我们定义目标哈密顿量为：
\[
H = H_f + \sum_{j=0}^{m-1} \lambda_j \xi(H_j)
\]
- \( H_f \): 目标函数哈密顿量，表示 \( C_{\max } \)：
  \[
  H_f = \sum_{h=0}^{l-1} 2^h \frac{1 - Z_h}{2}
  \]
- \( H_j \): 约束哈密顿量，表示 \( h_j(x, z) \leq 0 \):
  \[
  H_j = \sum_{i=0}^{n-1} p_{ij} \frac{1 - Z_{ij}}{2} - \sum_{h=0}^{l-1} 2^h \frac{1 - Z_h}{2}
  \]
- \( \xi(H_j) \): 定制惩罚函数，作用于 \( H_j \) 的本征值。两种选择：
  - **Heaviside 阶跃函数**：
    \[
    \xi(\mu_j) = \delta(\mu_j) = \begin{cases} 
    1 & \text{if } \mu_j > 0 \\
    0 & \text{if } \mu_j \leq 0 
    \end{cases}
    \]
    仅对违反约束的解 (\( C_j > C_{\max } \)) 施加惩罚。
  - **指数函数**：
    \[
    \xi(\mu_j) = e^{\lambda_2 \mu_j}
    \]
    对违反约束的解施加指数增长的惩罚，近似理想约束分离。
- \( \lambda_j \): 惩罚系数，确保可行解优先。建议 \( \lambda_j > \lambda_{\text{UB}} \)，其中 \( \lambda_{\text{UB}} \) 是目标函数的上界（如 \( \sum_{i,j} p_{ij} \)).

此外，One-hot 约束 \( \sum_{j=0}^{m-1} x_{ij} = 1 \) 通过二次惩罚项纳入：
\[
H_{\text{one-hot}} = K_1 \sum_{i=0}^{n-1} \left( \sum_{j=0}^{m-1} \frac{1 - Z_{ij}}{2} - 1 \right)^2
\]
总哈密顿量为：
\[
H = H_f + H_{\text{one-hot}} + \sum_{j=0}^{m-1} \lambda_j \xi(H_j)
\]

---

### 3. 期望值计算
由于 \( \xi(H_j) \) 是非线性函数，无法直接作为可观测值测量。采用混合量子-经典计算：
- **量子部分**：生成变分态 \( |\psi(\theta)\rangle \)，测量 Pauli-Z 项，获取概率分布 \( p_j = |\langle j | \psi(\theta) \rangle|^2 \)。
- **经典部分**：计算非线性期望值：
  \[
  \langle \xi(H_j) \rangle = \sum_{j=0}^{2^{N_q}-1} \xi(\mu_j) p_j
  \]
  其中 \( \mu_j \) 是 \( H_j \) 的本征值：
  \[
  H_j |j\rangle = \mu_j |j\rangle
  \]
  计算 \( \mu_j \):
  \[
  \mu_j = \sum_{i=0}^{n-1} p_{ij} \frac{1 - (-1)^{j_{ij}}}{2} - \sum_{h=0}^{l-1} 2^h \frac{1 - (-1)^{j_h}}{2}
  \]
  其中 \( j_{ij}, j_h \in \{0, 1\} \) 是基态 \( |j\rangle \) 的比特值（0 对应 1，1 对应 -1）。

总期望值：
\[
\langle H \rangle = \langle H_f \rangle + \langle H_{\text{one-hot}} \rangle + \sum_{j=0}^{m-1} \lambda_j \langle \xi(H_j) \rangle
\]
- \( \langle H_f \rangle, \langle H_{\text{one-hot}} \rangle \): 通过量子电路测量 Pauli-Z 项。
- \( \langle \xi(H_j) \rangle \): 经典计算，复杂度为 \( \mathcal{O}(2^{n+1}) \)（因 \( H_j \) 涉及 \( n+1 \) 个非零项）。

---

### 4. VQE 实现流程
VQE 通过最小化 \( \langle H \rangle \) 逼近最优解。流程如下：

#### 4.1 初始化
- 使用 \( N_q = n \times m + l \) 个量子比特，应用 Hadamard 门生成均匀叠加态：
  \[
  |+\rangle^{\otimes N_q} = \frac{1}{\sqrt{2^{N_q}}} \sum_{x \in \{0, 1\}^{N_q}} |x\rangle
  \]

#### 4.2 变分量子态
- 使用硬件高效 ansatz（HEA）：
  - 旋转层：\( R_Y(\theta_{k,i}) \) 门。
  - 纠缠层：相邻量子比特的 \( CZ \) 门。
- \( p \)-层 ansatz：
  \[
  |\psi(\theta)\rangle = \prod_{k=0}^{p-1} U_{\text{ent}} U_{\text{rot}}(\theta_k) |+\rangle^{\otimes N_q}
  \]

#### 4.3 期望值计算
- **量子测量**：
  - 测量 \( H_f \):
    \[
    \langle H_f \rangle = \sum_{h=0}^{l-1} 2^h \frac{1 - \langle Z_h \rangle}{2}
    \]
  - 测量 \( H_{\text{one-hot}} \):
    \[
    \langle H_{\text{one-hot}} \rangle = K_1 \sum_{i=0}^{n-1} \left\langle \left( \sum_{j=0}^{m-1} \frac{1 - Z_{ij}}{2} - 1 \right)^2 \right\rangle
    \]
  - 获取概率分布 \( p_j \)。
- **经典计算**：
  - 计算 \( \mu_j \)，应用 \( \xi(\mu_j) = \delta(\mu_j) \) 或 \( e^{\lambda_2 \mu_j} \)。
  - 求和：
    \[
    \langle \xi(H_j) \rangle = \sum_{j=0}^{2^{N_q}-1} \xi(\mu_j) p_j
    \]

#### 4.4 经典优化
- 使用 Powell 优化器调整 \( \theta \)，最小化 \( \langle H \rangle \)。
- 优化目标：
  \[
  \min_{\theta} \langle \psi(\theta) | H | \psi(\theta) \rangle
  \]

#### 4.5 结果提取
- 采样优化后的 \( |\psi(\theta)\rangle \)，获取最优 \( x_{ij}, z_h \)。
- 通过 Gantt 图验证调度结果。

---

### 5. 实验设置
与文章实验环境一致：

- **问题实例**：
  - \( n = 4, m = 3 \)。
  - 处理时间矩阵：
    \[
    p_{ij} = \begin{bmatrix}
    1 & 1 & 2 \\
    3 & 1 & 1 \\
    2 & 1 & 1 \\
    2 & 2 & 2
    \end{bmatrix}
    \]
  - 最优解：\( C_{\max } = 3 \)，分配为：作业 0 → 机器 1，作业 1 → 机器 2，作业 2 → 机器 0，作业 3 → 机器 1。

- **量子比特需求**：
  - One-hot 编码：\( n \times m = 12 \) 位。
  - \( C_{\max } \): \( l = \lceil \log_2 (3 \cdot 4) \rceil = 4 \) 位。
  - 总计：\( N_q = 12 + 4 = 16 \)，比原方法（20 位）减少 4 位。

- **实验环境**：
  - 硬件：Intel Core CPU，2.20GHz，32GB 内存。
  - 软件：Qiskit，`qasm_simulator` 模拟噪声。
  - 优化器：SciPy 的 Powell。
  - VQE 参数：HEA，\( p = 1, 2, 3 \)，随机种子 10，1000 次采样，运行 10 次。

- **惩罚系数**：
  - \( K_1 \): One-hot 约束，设 \( K_1 > \sum_{i,j} p_{ij} = 22 \)，如 \( K_1 = 50 \)。
  - \( \lambda_j \): 约束惩罚，设 \( \lambda_j > \sum_{i,j} p_{ij} \)，如 \( \lambda_j = 50 \)。
  - 指数惩罚：\( \lambda_2 = 1, 10, 50 \) 测试。

- **评估指标**：
  - 最优解概率 \( P_{\text{opt}} \):
    \[
    P_{\text{opt}} = \frac{\text{满足最优 } C_{\max } \text{ 且无惩罚的样本数}}{\text{总样本数}}
    \]
  - 量子比特需求。
  - Gantt 图验证。

---

### 6. 预期结果与分析
- **性能**：
  - 相比 OH-HC，定制惩罚函数（OH-CPF，One-hot + Custom Penalty Function）减少量子比特（16 vs. 20），提高资源效率。
  - Heaviside 惩罚明确区分可行与不可行解，预期 \( P_{\text{opt}} \) 优于 OH-HC（0.14 at \( p=3 \））和 Monta nez-Barrera 的二次近似（因避免虚假最小值）。
  - 指数惩罚随 \( \lambda_2 \) 增大，可行性提升，但最优性可能下降（与 Lee & Lau 的 MKP 结果一致）。

- **与原方法的比较**：
  - OH-DE 使用 \( \max(0, C_j - z) \)，类似 Heaviside，但需经典计算。OH-CPF 直接用 \( \delta(H_j) \)，逻辑更清晰。
  - BIN-DE 使用二进制编码，量子比特更少（约 10 位），但需额外无效分配惩罚。OH-CPF 保留 One-hot 的简单性，适合小规模实例。

- **Gantt 图验证**：
  - 验证 \( C_{\max } = 3 \)，机器 1: \( p_{01} + p_{31} = 2 \)，机器 2: \( p_{12} = 1 \)，机器 0: \( p_{20} = 2 \)，均满足约束。

---

### 7. 代码实现（伪代码）
以下是 VQE 实现 FOSSP 的代码，使用 Heaviside 惩罚函数：

In [1]:
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.circuit.library import TwoLocal, RYGate, CXGate
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns

# FOSSP问题实例参数
n, m = 4, 3  # n个作业，m台机器
p_ij = np.array([[1, 1, 2],   # 处理时间矩阵：每行代表一个作业，每列代表一台机器
                 [3, 1, 1],   # p_ij[i,j]表示作业i在机器j上的处理时间
                 [2, 1, 1],
                 [2, 2, 2]])
l = 4  # 用于表示C_max的二进制位数
Nq = n * m + l  # 总量子比特数 = One-hot编码所需比特 + C_max表示所需比特
K1 = 50  # One-hot约束的惩罚系数
lambda_j = 50  # 约束条件的惩罚系数

# Heaviside阶跃函数：用于惩罚违反约束的解
def heaviside(mu):
    """
    实现Heaviside阶跃函数
    mu > 0时返回1（违反约束），否则返回0（满足约束）
    """
    return 1 if mu > 0 else 0

# 计算约束哈密顿量H_j的本征值
def compute_mu_j(state, j):
    """
    计算机器j的完成时间与C_max的差值
    state: 量子态的二进制表示
    j: 机器索引
    返回: C_j - C_max，正值表示违反约束
    """
    # 提取One-hot编码的作业分配
    x = [int(state[i * m + k]) for i in range(n) for k in range(m)]
    # 提取C_max的二进制表示
    z = [int(state[n * m + h]) for h in range(l)]
    # 计算机器j的完成时间
    C_j = sum(x[i * m + j] * p_ij[i, j] for i in range(n))
    # 计算C_max
    C_max = sum(2**h * z[h] for h in range(l))
    return C_j - C_max

# 目标函数组件：计算C_max的期望值
def compute_H_f(counts):
    """
    计算目标函数哈密顿量H_f的期望值
    counts: 量子态测量结果的计数字典
    """
    energy = 0
    for bitstring, count in counts.items():
        z = [int(bitstring[n * m + h]) for h in range(l)]
        energy += sum(2**h * (1 - (-1)**z[h]) / 2 for h in range(l)) * count / 1000
    return energy

# 计算One-hot约束的惩罚项
def compute_H_one_hot(counts):
    """
    计算One-hot约束的惩罚能量
    确保每个作业只分配给一台机器
    """
    energy = 0
    for bitstring, count in counts.items():
        x = [int(bitstring[i * m + j]) for i in range(n) for j in range(m)]
        # 计算每个作业的分配是否满足One-hot约束
        penalty = sum((sum(x[i * m + j] for j in range(m)) - 1)**2 for i in range(n))
        energy += K1 * penalty * count / 1000
    return energy

# 计算约束惩罚项
def compute_xi_H_j(counts):
    """
    计算所有机器的约束惩罚项
    使用Heaviside函数惩罚违反C_j ≤ C_max的解
    """
    xi_energy = [0] * m
    for j in range(m):
        for bitstring, count in counts.items():
            mu_j = compute_mu_j(bitstring, j)
            xi_energy[j] += heaviside(mu_j) * count / 1000
    return sum(lambda_j * xi_energy[j] for j in range(m))

# 创建VQE电路
def create_ansatz(p):
    """
    创建变分量子本征求解器的电路
    p: 电路深度（重复层数）
    """
    # 创建主电路：使用RY旋转门和CX纠缠门
    ansatz = TwoLocal(Nq, rotation_blocks='ry', entanglement_blocks='cx', 
                     entanglement='linear', reps=p)
    # 添加测量电路
    meas = QuantumCircuit(Nq, Nq)
    meas.measure_all()
    # 组合主电路和测量电路
    complete_circuit = ansatz.compose(meas)
    # 转换为基本量子门
    basic_circuit = transpile(complete_circuit, basis_gates=['rx', 'ry', 'rz', 'cx'], 
                            optimization_level=1)
    return basic_circuit

# 计算期望值
def compute_expectation(params, ansatz, backend):
    """
    计算给定参数下的哈密顿量期望值
    params: 变分参数
    ansatz: 量子电路
    backend: 量子模拟器后端
    """
    try:
        # 将参数赋值到电路
        circ = ansatz.assign_parameters(params)
        # 优化电路
        circ = transpile(circ, backend=backend, basis_gates=['rx', 'ry', 'rz', 'cx'], 
                        optimization_level=1)
        # 执行量子电路
        job = backend.run(circ, shots=1000)
        counts = job.result().get_counts()
        # 计算总能量
        H_f = compute_H_f(counts)
        H_one_hot = compute_H_one_hot(counts)
        xi_H_j = compute_xi_H_j(counts)
        return H_f + H_one_hot + xi_H_j
    except Exception as e:
        print(f"计算期望值时出错: {str(e)}")
        return float('inf')  # 出错时返回无穷大作为惩罚

# VQE优化
def vqe_optimize(p, num_trials=5):
    """
    执行VQE优化
    p: 电路深度
    num_trials: 重复优化次数
    返回: 最佳结果
    """
    best_result = None
    best_energy = float('inf')
    
    for trial in range(num_trials):
        ansatz = create_ansatz(p)
        backend = Aer.get_backend('qasm_simulator')
        init_params = np.random.rand(ansatz.num_parameters) * 2 * np.pi
        result = minimize(compute_expectation, init_params, 
                        args=(ansatz, backend), method='Powell',
                        options={'maxiter': 100})
        
        if result.fun < best_energy:
            best_energy = result.fun
            best_result = result
            
    return best_result


In [ ]:
# 运行实验并收集结果
def run_experiments(p_values=[1, 2, 3], num_trials=5):
    """
    执行多组实验并收集结果
    p_values: 要测试的电路深度列表
    num_trials: 每个深度的重复次数
    """
    results = []
    for p in p_values:
        print(f"\n正在优化深度 p={p} 的电路...")
        result = vqe_optimize(p, num_trials)
        results.append({
            'p': p,
            'energy': result.fun,
            'success': result.success,
            'nfev': result.nfev,
            'parameters': result.x
        })
        print(f"p={p}, 能量: {result.fun:.3f}, 成功: {result.success}")
    return results

# 绘制甘特图
def plot_gantt(best_counts):
    """
    根据最优解绘制甘特图
    best_counts: 最优量子态的测量结果
    """
    # 找到最频繁出现的状态
    best_state = max(best_counts.items(), key=lambda x: x[1])[0]
    
    # 解析状态获取作业分配
    x = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            x[i, j] = int(best_state[i * m + j])
    
    # 创建甘特图
    fig, ax = plt.subplots(figsize=(12, 6))
    colors = plt.cm.Set3(np.linspace(0, 1, n))
    
    for j in range(m):  # 对每台机器
        current_time = 0
        for i in range(n):  # 对每个作业
            if x[i, j] == 1:  # 如果作业i分配给机器j
                ax.barh(j, p_ij[i, j], left=current_time, color=colors[i],
                       label=f'作业 {i}' if j == 0 else "")
                current_time += p_ij[i, j]
    
    ax.set_yticks(range(m))
    ax.set_yticklabels([f'机器 {j}' for j in range(m)])
    ax.set_xlabel('时间')
    ax.set_title('调度甘特图')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

# 绘制能量收敛图
def plot_energy_convergence(results):
    """
    绘制不同电路深度的能量收敛图
    """
    plt.figure(figsize=(10, 6))
    energies = [result['energy'] for result in results]
    depths = [result['p'] for result in results]
    
    plt.plot(depths, energies, 'bo-')
    plt.scatter(depths, energies, color='blue', s=100)
    
    plt.xlabel('电路深度 (p)')
    plt.ylabel('能量')
    plt.title('能量随电路深度的变化')
    plt.grid(True)
    plt.show()

# 分析最优解
def analyze_solution(best_result, ansatz):
    """
    分析最优解的详细信息
    """
    backend = Aer.get_backend('qasm_simulator')
    circ = ansatz.assign_parameters(best_result['parameters'])
    circ = transpile(circ, backend=backend, basis_gates=['rx', 'ry', 'rz', 'cx'], 
                    optimization_level=1)
    job = backend.run(circ, shots=1000)
    counts = job.result().get_counts()
    
    # 找到最频繁的状态
    best_state = max(counts.items(), key=lambda x: x[1])[0]
    
    # 解析作业分配
    x = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            x[i, j] = int(best_state[i * m + j])
    
    # 解析C_max
    z = [int(best_state[n * m + h]) for h in range(l)]
    C_max = sum(2**h * z[h] for h in range(l))
    
    # 计算每台机器的完成时间
    machine_times = np.zeros(m)
    for j in range(m):
        for i in range(n):
            if x[i, j] == 1:
                machine_times[j] += p_ij[i, j]
    
    print("\n最优解分析:")
    print(f"C_max = {C_max}")
    print("\n作业分配:")
    for i in range(n):
        machine = np.where(x[i] == 1)[0][0]
        print(f"作业 {i} → 机器 {machine}")
    print("\n机器完成时间:")
    for j in range(m):
        print(f"机器 {j}: {machine_times[j]}")

# 运行实验
print("开始运行FOSSP-VQE实验...")
results = run_experiments(p_values=[1, 2, 3], num_trials=5)

# 绘制能量收敛图
print("\n绘制能量收敛图...")
plot_energy_convergence(results)

# 找到最佳结果
best_result = min(results, key=lambda x: x['energy'])
print(f"\n最佳结果:")
print(f"电路深度 p = {best_result['p']}")
print(f"能量 = {best_result['energy']:.3f}")
print(f"函数评估次数 = {best_result['nfev']}")

# 使用最佳参数重新运行电路以获取状态分布
best_ansatz = create_ansatz(best_result['p'])
analyze_solution(best_result, best_ansatz)

# 绘制最优解的甘特图
print("\n绘制最优调度的甘特图...")


开始运行FOSSP-VQE实验...

正在优化深度 p=1 的电路...


C:\Users\李华宪\AppData\Local\Temp\ipykernel_21828\534542451.py:142: DeprecationWarning: The 'qiskit.Aer' entry point is deprecated and will be removed in Qiskit 1.0. You should use 'qiskit_aer.Aer' directly instead.
  backend = Aer.get_backend('qasm_simulator')


p=1, 能量: 6.238, 成功: True

正在优化深度 p=2 的电路...
p=2, 能量: 12.661, 成功: True

正在优化深度 p=3 的电路...


In [6]:
# Run VQE
for p in [1, 2, 3]:
    result = vqe_optimize(p)
    print(f"p={p}, Energy: {result.fun}, Success: {result.success}")

# Gantt chart visualization (simplified)
def plot_gantt(counts):
    # Extract optimal assignment and plot (omitted for brevity)
    pass

p=1, Energy: 16.186999999999998, Success: True
p=2, Energy: 11.243999999999991, Success: True
p=3, Energy: 23.131999999999998, Success: True
